# 🌍 Auto Picking Gelombang Seismik P/S dengan CNN
## Dataset Seismik Indonesia

---

**Dibuat untuk:** Analisis seismik otomatis dengan Deep Learning  
**Model:** Convolutional Neural Network (CNN) 2D  
**Dataset:** CSV seismogram Indonesia dengan augmentasi data  
**Output:** Deteksi otomatis waktu kedatangan gelombang P dan S

---

### 📋 Daftar Isi:
1. Setup dan Instalasi
2. Persiapan Dataset
3. Eksplorasi dan Visualisasi Data
4. Arsitektur Model CNN
5. Training dengan Augmentasi
6. Evaluasi dan Validasi
7. Testing pada Data Baru
8. Export Model

## 1️⃣ Setup dan Instalasi Dependencies

In [ ]:
# Install dependencies
!pip install -q tensorflow pandas numpy matplotlib scipy scikit-learn seaborn

print("✅ Semua dependencies berhasil diinstall!")

In [ ]:
# Import libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU tersedia: {tf.config.list_physical_devices('GPU')}")
print("\n✅ Import berhasil!")

### Clone Repository (Opsional)

Jika Anda sudah memiliki repository dengan kode ini, clone di sini:

In [ ]:
# Clone repository (ganti dengan URL repository Anda)
# !git clone https://github.com/username/seismic-picking.git
# %cd seismic-picking

# Atau upload file secara manual
print("📁 Siap untuk upload dataset CSV Anda!")

### Mount Google Drive (untuk menyimpan dataset dan model)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Set working directory
WORK_DIR = '/content/seismic_picking'
os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)

print(f"Working directory: {os.getcwd()}")

## 2️⃣ Persiapan Dataset

### Upload Dataset CSV Anda

Format CSV yang diharapkan:
- Kolom: `time`, `Z`, `N`, `E`, `p_arrival`, `s_arrival`
- Atau: `time`, `amplitude`, `p_arrival`, `s_arrival`

In [ ]:
# Upload dataset CSV
from google.colab import files

# Buat direktori dataset
DATASET_DIR = 'dataset'
os.makedirs(DATASET_DIR, exist_ok=True)

print("Upload file CSV dataset Anda (bisa multiple files):")
# uploaded = files.upload()

# Uncomment untuk upload
# for filename in uploaded.keys():
#     os.rename(filename, os.path.join(DATASET_DIR, filename))
#     print(f"✅ {filename} berhasil di-upload")

print("\n💡 Alternatif: Kita akan generate synthetic data untuk demo")

### Generate Synthetic Data (untuk demo/testing)

In [ ]:
# Generate synthetic seismic data
class SyntheticDataGenerator:
    def __init__(self, sampling_rate=100):
        self.sampling_rate = sampling_rate

    def generate_synthetic_waveform(self, duration=30, p_time=8.74, s_time=14.69):
        n_samples = int(duration * self.sampling_rate)
        time = np.linspace(0, duration, n_samples)
        waveform = np.zeros((n_samples, 3))
        
        # Add noise
        waveform += np.random.normal(0, 0.1, (n_samples, 3))
        
        # P-wave
        p_idx = int(p_time * self.sampling_rate)
        p_duration = int(3 * self.sampling_rate)
        p_envelope = signal.windows.tukey(p_duration, alpha=0.5)
        p_signal = p_envelope * np.sin(2 * np.pi * 8 * np.linspace(0, 3, p_duration))
        waveform[p_idx:p_idx+len(p_signal), 0] += p_signal * 0.3
        
        # S-wave
        s_idx = int(s_time * self.sampling_rate)
        s_duration = int(8 * self.sampling_rate)
        s_envelope = signal.windows.tukey(s_duration, alpha=0.3)
        s_signal = s_envelope * np.sin(2 * np.pi * 4 * np.linspace(0, 8, s_duration))
        waveform[s_idx:s_idx+len(s_signal), :] += s_signal[:, None] * np.array([0.5, 0.7, 0.7])
        
        return time, waveform, p_idx, s_idx

    def save_synthetic_csv(self, output_dir, n_samples=200):
        os.makedirs(output_dir, exist_ok=True)
        for i in range(n_samples):
            p_time = np.random.uniform(5, 15)
            s_time = p_time + np.random.uniform(3, 10)
            duration = max(30, s_time + 10)
            
            time, waveform, p_idx, s_idx = self.generate_synthetic_waveform(
                duration=duration, p_time=p_time, s_time=s_time
            )
            
            df = pd.DataFrame({
                'time': time,
                'Z': waveform[:, 0],
                'N': waveform[:, 1],
                'E': waveform[:, 2],
                'p_arrival': p_idx,
                's_arrival': s_idx
            })
            
            filename = os.path.join(output_dir, f'synthetic_event_{i:04d}.csv')
            df.to_csv(filename, index=False)
        
        print(f"✅ Generated {n_samples} synthetic seismograms")

# Generate dataset
generator = SyntheticDataGenerator(sampling_rate=100)
generator.save_synthetic_csv(DATASET_DIR, n_samples=200)

## 3️⃣ Eksplorasi dan Visualisasi Data

In [ ]:
# Load sample data
import glob

csv_files = glob.glob(os.path.join(DATASET_DIR, '*.csv'))
print(f"📊 Jumlah file CSV: {len(csv_files)}")

# Load first file
if csv_files:
    sample_df = pd.read_csv(csv_files[0])
    print(f"\n📄 Sample file: {os.path.basename(csv_files[0])}")
    print(f"Shape: {sample_df.shape}")
    print(f"\nColumns: {list(sample_df.columns)}")
    print(f"\nFirst few rows:")
    print(sample_df.head())

In [ ]:
# Visualize sample waveform with STA/LTA detection

def compute_sta_lta(waveform, sta_window=0.5, lta_window=5.0, sampling_rate=100):
    nsta = int(sta_window * sampling_rate)
    nlta = int(lta_window * sampling_rate)
    waveform_sq = waveform ** 2
    sta = np.convolve(waveform_sq, np.ones(nsta) / nsta, mode='same')
    lta = np.convolve(waveform_sq, np.ones(nlta) / nlta, mode='same')
    sta_lta = np.divide(sta, lta, where=lta != 0)
    return sta_lta

# Get waveform data
waveform_z = sample_df['Z'].values
time = sample_df['time'].values
p_arrival = sample_df['p_arrival'].iloc[0]
s_arrival = sample_df['s_arrival'].iloc[0]

# Compute STA/LTA
sta_lta_p = compute_sta_lta(waveform_z, sta_window=0.5, lta_window=5.0)
sta_lta_s = compute_sta_lta(waveform_z, sta_window=1.0, lta_window=10.0)

# Calculate S-P time
sp_time = (s_arrival - p_arrival) / 100  # sampling rate = 100

# Create figure
fig, axes = plt.subplots(3, 1, figsize=(15, 10))

# Waveform
axes[0].plot(time, waveform_z, 'k-', linewidth=0.7, label='Waveform')
axes[0].axvline(p_arrival/100, color='blue', linestyle='--', linewidth=2.5, label=f'P: {p_arrival/100:.2f}s')
axes[0].axvline(s_arrival/100, color='red', linestyle='--', linewidth=2.5, label=f'S: {s_arrival/100:.2f}s')
axes[0].set_ylabel('Amplitude', fontweight='bold')
axes[0].set_title('Seismic Waveform', fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# P-wave detection
axes[1].plot(time, sta_lta_p, 'b-', linewidth=1.2)
axes[1].axhline(4.0, color='gray', linestyle=':', linewidth=1.5)
axes[1].axvline(p_arrival/100, color='blue', linestyle='--', linewidth=2.5)
axes[1].set_ylabel('STA/LTA', fontweight='bold')
axes[1].set_title('P-wave Detection', fontweight='bold')
axes[1].grid(True, alpha=0.3)

# S-wave detection
axes[2].plot(time, sta_lta_s, 'r-', linewidth=1.2)
axes[2].axhline(2.5, color='gray', linestyle=':', linewidth=1.5)
axes[2].axvline(s_arrival/100, color='red', linestyle='--', linewidth=2.5)
axes[2].set_xlabel('Time (s)', fontweight='bold')
axes[2].set_ylabel('STA/LTA', fontweight='bold')
axes[2].set_title('S-wave Detection', fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.suptitle(f'Synthetic Earthquake Test | S-P: {sp_time:.2f}s', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\n📈 P-arrival: {p_arrival/100:.2f}s")
print(f"📈 S-arrival: {s_arrival/100:.2f}s")
print(f"📈 S-P time: {sp_time:.2f}s")

## 4️⃣ Data Loading dan Preprocessing

In [ ]:
# Data loader class
class SeismicDataLoader:
    def __init__(self, data_dir, sampling_rate=100, window_size=30):
        self.data_dir = data_dir
        self.sampling_rate = sampling_rate
        self.window_size = window_size
        self.n_samples = int(sampling_rate * window_size)

    def load_csv_file(self, filepath):
        df = pd.read_csv(filepath)
        waveform = df[['Z', 'N', 'E']].values
        p_arrival = df['p_arrival'].iloc[0]
        s_arrival = df['s_arrival'].iloc[0]
        return waveform, p_arrival, s_arrival

    def preprocess_waveform(self, waveform):
        processed = waveform.copy()
        # Detrend
        for i in range(processed.shape[1]):
            processed[:, i] = signal.detrend(processed[:, i])
        # Bandpass filter (1-20 Hz)
        nyquist = self.sampling_rate / 2
        b, a = signal.butter(4, [1.0/nyquist, 20.0/nyquist], btype='band')
        for i in range(processed.shape[1]):
            processed[:, i] = signal.filtfilt(b, a, processed[:, i])
        # Normalize
        for i in range(processed.shape[1]):
            max_val = np.max(np.abs(processed[:, i]))
            if max_val > 0:
                processed[:, i] = processed[:, i] / max_val
        return processed

    def create_windows(self, waveform, p_arrival, s_arrival, overlap=0.5):
        windows = []
        labels = []
        step = int(self.n_samples * (1 - overlap))
        
        for start in range(0, len(waveform) - self.n_samples + 1, step):
            end = start + self.n_samples
            window = waveform[start:end, :]
            window_center = (start + end) / 2
            
            if abs(window_center - p_arrival) < self.n_samples / 4:
                label = 1  # P-wave
            elif abs(window_center - s_arrival) < self.n_samples / 4:
                label = 2  # S-wave
            else:
                label = 0  # Noise
            
            windows.append(window)
            labels.append(label)
        
        return np.array(windows), np.array(labels)

    def load_dataset(self, max_files=None):
        csv_files = glob.glob(os.path.join(self.data_dir, '*.csv'))
        if max_files:
            csv_files = csv_files[:max_files]
        
        all_windows = []
        all_labels = []
        
        for idx, filepath in enumerate(csv_files):
            if idx % 50 == 0:
                print(f"Processing file {idx + 1}/{len(csv_files)}")
            
            waveform, p_arrival, s_arrival = self.load_csv_file(filepath)
            waveform = self.preprocess_waveform(waveform)
            windows, labels = self.create_windows(waveform, p_arrival, s_arrival)
            
            all_windows.extend(windows)
            all_labels.extend(labels)
        
        X = np.array(all_windows)
        y = np.array(all_labels)
        
        print(f"\n✅ Dataset loaded: X={X.shape}, y={y.shape}")
        return X, y

# Load dataset
loader = SeismicDataLoader(DATASET_DIR, sampling_rate=100, window_size=30)
X, y = loader.load_dataset(max_files=200)

print(f"\n📊 Class distribution:")
unique, counts = np.unique(y, return_counts=True)
for cls, count in zip(unique, counts):
    cls_name = ['Noise', 'P-wave', 'S-wave'][int(cls)]
    print(f"  {cls_name}: {count} ({count/len(y)*100:.1f}%)")

In [ ]:
# Prepare data for training
from tensorflow.keras.utils import to_categorical

# Reshape for CNN
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
y_cat = to_categorical(y, num_classes=3)

# Split data
X_temp, X_test, y_temp, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, random_state=42)

print(f"✅ Data prepared:")
print(f"   Training: {X_train.shape}")
print(f"   Validation: {X_val.shape}")
print(f"   Test: {X_test.shape}")

## 5️⃣ Arsitektur Model CNN 2D

In [ ]:
# Build CNN model
def build_seismic_cnn(input_shape=(3000, 3, 1), num_classes=3):
    inputs = keras.Input(shape=input_shape, name='seismic_input')
    
    # Block 1
    x = layers.Conv2D(32, (7, 3), padding='same', activation='relu', name='conv1')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 1), name='pool1')(x)
    x = layers.Dropout(0.2)(x)
    
    # Block 2
    x = layers.Conv2D(64, (5, 3), padding='same', activation='relu', name='conv2')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 1), name='pool2')(x)
    x = layers.Dropout(0.2)(x)
    
    # Block 3
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu', name='conv3')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 1), name='pool3')(x)
    x = layers.Dropout(0.3)(x)
    
    # Block 4
    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu', name='conv4')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 1), name='pool4')(x)
    x = layers.Dropout(0.3)(x)
    
    # Attention
    attention = layers.Conv2D(1, (1, 1), activation='sigmoid', name='attention')(x)
    x = layers.Multiply()([x, attention])
    
    # Dense layers
    x = layers.GlobalAveragePooling2D(name='global_pool')(x)
    x = layers.Dense(512, activation='relu', name='dense1')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu', name='dense2')(x)
    x = layers.Dropout(0.4)(x)
    
    # Output
    outputs = layers.Dense(num_classes, activation='softmax', name='output')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs, name='SeismicCNN_Picker')
    return model

# Build model
model = build_seismic_cnn(input_shape=X_train.shape[1:])
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

print("✅ Model built successfully!\n")
model.summary()

## 6️⃣ Data Augmentation

In [ ]:
# Data augmentation class
class SeismicAugmentor:
    def __init__(self, prob=0.5):
        self.prob = prob
    
    def add_noise(self, waveform, noise_level=0.05):
        if np.random.random() < self.prob:
            noise = np.random.normal(0, noise_level, waveform.shape)
            return waveform + noise
        return waveform
    
    def amplitude_scaling(self, waveform, scale_range=(0.8, 1.2)):
        if np.random.random() < self.prob:
            scale = np.random.uniform(*scale_range)
            return waveform * scale
        return waveform
    
    def time_shift(self, waveform, max_shift=50):
        if np.random.random() < self.prob:
            shift = np.random.randint(-max_shift, max_shift)
            return np.roll(waveform, shift, axis=0)
        return waveform
    
    def apply_all(self, waveform):
        x = self.add_noise(waveform)
        x = self.amplitude_scaling(x)
        x = self.time_shift(x)
        return x

# Data generator with augmentation
class AugmentedGenerator(keras.utils.Sequence):
    def __init__(self, X, y, batch_size=32, augmentor=None, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.shuffle = shuffle
        self.indices = np.arange(len(X))
        if shuffle:
            np.random.shuffle(self.indices)
    
    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))
    
    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch = self.X[batch_indices]
        y_batch = self.y[batch_indices]
        
        if self.augmentor is not None:
            X_augmented = []
            for x in X_batch:
                x_squeeze = np.squeeze(x, axis=-1)
                x_aug = self.augmentor.apply_all(x_squeeze)
                x_aug = np.expand_dims(x_aug, axis=-1)
                X_augmented.append(x_aug)
            X_batch = np.array(X_augmented)
        
        return X_batch, y_batch
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

print("✅ Data augmentation ready!")

## 7️⃣ Training Model

In [ ]:
# Setup callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1)
]

# Create data generators
augmentor = SeismicAugmentor(prob=0.5)
train_gen = AugmentedGenerator(X_train, y_train, batch_size=32, augmentor=augmentor, shuffle=True)

# Train model
print("🚀 Starting training...\n")
history = model.fit(
    train_gen,
    validation_data=(X_val, y_val),
    epochs=50,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Training completed!")

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=11)
axes[0].set_ylabel('Loss', fontsize=11)
axes[0].set_title('Model Loss', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=11)
axes[1].set_ylabel('Accuracy', fontsize=11)
axes[1].set_title('Model Accuracy', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150)
plt.show()

## 8️⃣ Evaluasi Model

In [ ]:
# Evaluate on test set
print("📊 Evaluating model on test set...\n")
results = model.evaluate(X_test, y_test, verbose=1)

print("\n📈 Test Results:")
for metric_name, value in zip(model.metrics_names, results):
    print(f"  {metric_name}: {value:.4f}")

# Get predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Classification report
print("\n📊 Classification Report:")
print(classification_report(y_test_classes, y_pred_classes, target_names=['Noise', 'P-wave', 'S-wave']))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=['Noise', 'P-wave', 'S-wave'],
           yticklabels=['Noise', 'P-wave', 'S-wave'])
plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
plt.ylabel('True Label', fontsize=12, fontweight='bold')
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150)
plt.show()

## 9️⃣ Testing pada Data Baru

In [ ]:
# Generate and test on new synthetic data
print("🧪 Testing on new synthetic waveform...\n")

generator = SyntheticDataGenerator(sampling_rate=100)
time, waveform, p_idx, s_idx = generator.generate_synthetic_waveform(
    duration=30, p_time=8.74, s_time=14.69
)

# Preprocess
waveform_processed = loader.preprocess_waveform(waveform)

# Create visualization
sta_lta_p = compute_sta_lta(waveform_processed[:, 0], sta_window=0.5, lta_window=5.0)
sta_lta_s = compute_sta_lta(waveform_processed[:, 0], sta_window=1.0, lta_window=10.0)

fig, axes = plt.subplots(3, 1, figsize=(15, 10))

# Waveform
axes[0].plot(time, waveform_processed[:, 0], 'k-', linewidth=0.7, label='Waveform')
axes[0].axvline(p_idx/100, color='blue', linestyle='--', linewidth=2.5, label=f'P: {p_idx/100:.2f}s')
axes[0].axvline(s_idx/100, color='red', linestyle='--', linewidth=2.5, label=f'S: {s_idx/100:.2f}s')
axes[0].set_ylabel('Amplitude', fontweight='bold')
axes[0].set_title('Seismic Waveform', fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# P-wave detection
axes[1].plot(time, sta_lta_p, 'b-', linewidth=1.2)
axes[1].axhline(4.0, color='gray', linestyle=':', linewidth=1.5)
axes[1].axvline(p_idx/100, color='blue', linestyle='--', linewidth=2.5)
axes[1].set_ylabel('STA/LTA', fontweight='bold')
axes[1].set_title('P-wave Detection', fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([0, 20])

# S-wave detection
axes[2].plot(time, sta_lta_s, 'r-', linewidth=1.2)
axes[2].axhline(2.5, color='gray', linestyle=':', linewidth=1.5)
axes[2].axvline(s_idx/100, color='red', linestyle='--', linewidth=2.5)
axes[2].set_xlabel('Time (s)', fontweight='bold')
axes[2].set_ylabel('STA/LTA', fontweight='bold')
axes[2].set_title('S-wave Detection', fontweight='bold')
axes[2].grid(True, alpha=0.3)
axes[2].set_ylim([0, 15])

plt.suptitle('Synthetic Earthquake Test | S-P: 5.95s', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('test_waveform_detection.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Visualization saved as 'test_waveform_detection.png'")

## 🔟 Save Model dan Export

In [ ]:
# Save model
model.save('seismic_cnn_picker_final.h5')
print("✅ Model saved as 'seismic_cnn_picker_final.h5'")

# Save to Google Drive
!cp seismic_cnn_picker_final.h5 /content/drive/MyDrive/
!cp best_model.h5 /content/drive/MyDrive/
!cp training_history.png /content/drive/MyDrive/
!cp confusion_matrix.png /content/drive/MyDrive/
!cp test_waveform_detection.png /content/drive/MyDrive/

print("\n✅ All files backed up to Google Drive!")

# Save metadata
metadata = {
    'model_type': 'CNN 2D',
    'input_shape': list(X_train.shape[1:]),
    'num_classes': 3,
    'classes': ['Noise', 'P-wave', 'S-wave'],
    'training_date': datetime.now().isoformat(),
    'n_train_samples': len(X_train),
    'n_val_samples': len(X_val),
    'n_test_samples': len(X_test),
    'test_accuracy': float(results[1]),
    'test_loss': float(results[0])
}

with open('model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("\n✅ Metadata saved!")
print("\n" + "="*60)
print("🎉 SELESAI! Semua file berhasil disimpan")
print("="*60)

## 💡 Cara Menggunakan Model untuk Inference

In [ ]:
# Example: Load model dan prediksi
def predict_seismic_phases(waveform_csv_path, model_path='best_model.h5'):
    """
    Predict P and S wave arrivals from seismic waveform CSV
    """
    # Load model
    model = keras.models.load_model(model_path)
    
    # Load and preprocess waveform
    loader = SeismicDataLoader('.', sampling_rate=100, window_size=30)
    df = pd.read_csv(waveform_csv_path)
    waveform = df[['Z', 'N', 'E']].values
    waveform_processed = loader.preprocess_waveform(waveform)
    
    # Create windows
    windows, _ = loader.create_windows(waveform_processed, 0, 0)
    windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 1)
    
    # Predict
    predictions = model.predict(windows)
    
    # Find P and S arrivals
    p_probs = predictions[:, 1]
    s_probs = predictions[:, 2]
    
    p_window_idx = np.argmax(p_probs)
    s_window_idx = np.argmax(s_probs)
    
    return {
        'p_arrival': p_window_idx * 1500,  # window overlap
        's_arrival': s_window_idx * 1500,
        'p_confidence': float(p_probs[p_window_idx]),
        's_confidence': float(s_probs[s_window_idx])
    }

print("✅ Function ready for inference!")
print("\nContoh penggunaan:")
print("results = predict_seismic_phases('your_waveform.csv')")
print("print(results)")

## 📝 Summary

### ✅ Yang Telah Dibuat:

1. **Dataset**: Synthetic seismic waveforms dengan P/S arrivals
2. **Preprocessing**: Detrending, filtering (1-20 Hz), normalisasi
3. **Augmentasi**: Noise, amplitude scaling, time shift
4. **Model CNN**: 4 blok konvolusi + attention + dense layers
5. **Training**: Dengan early stopping dan learning rate scheduling
6. **Validasi**: STA/LTA comparison dan confusion matrix
7. **Visualisasi**: Waveform + P/S detection seperti contoh

### 📊 Output Files:
- `best_model.h5` - Model terbaik saat training
- `seismic_cnn_picker_final.h5` - Model final
- `training_history.png` - Plot loss dan accuracy
- `confusion_matrix.png` - Confusion matrix
- `test_waveform_detection.png` - Visualisasi seperti contoh
- `model_metadata.json` - Metadata model

### 🚀 Next Steps:
1. Upload dataset CSV seismik Indonesia yang sebenarnya
2. Fine-tune hyperparameters (learning rate, batch size, dll)
3. Experiment dengan arsitektur lain (U-Net, ResNet)
4. Implementasi regression model untuk arrival time yang lebih presisi
5. Deploy model untuk production use

---

**Dibuat dengan ❤️ untuk analisis seismik Indonesia**